# Monte Carlo Samples of Halo Lightcones

Generate an example sample of host halos on a lightcone.

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
from jax import random as jran
ran_key = jran.key(0)

In [ ]:
from diffsky.experimental import mc_lightcone_halos as mclh

lgmp_min = 11.5
z_min, z_max = 0.1, 3.0
sky_area_degsq = 2.0
args = (ran_key, lgmp_min, z_min, z_max, sky_area_degsq)

redshift, lgmp = mclh.mc_lightcone_host_halo_mass_function(*args)

In [ ]:
fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 4))
yscale = ax1.set_yscale('log')

__=ax0.hist(redshift, bins=100, alpha=0.7)
__=ax1.hist(lgmp, bins=100, alpha=0.7)
xlabel = ax0.set_xlabel(r'${\rm redshift}$')
xlabel = ax1.set_xlabel(r'${\rm log_{10}M_{halo}}$')

## Check self-consistency of lightcone vs single-z Monte Carlo generators

Generate a halo lightcone over a narrow range of redshift, and compare the halo mass function to a Monte Carlo realization of a population at a single redshift.

In [ ]:
from dsps.cosmology import DEFAULT_COSMOLOGY, flat_wcdm

z_min, z_max = 0.5, 0.55

d_com_z_min = flat_wcdm.comoving_distance_to_z(z_min, *DEFAULT_COSMOLOGY)
d_com_z_max = flat_wcdm.comoving_distance_to_z(z_max, *DEFAULT_COSMOLOGY)

vol_z_min = (4/3)*np.pi*d_com_z_min**3
vol_z_max = (4/3)*np.pi*d_com_z_max**3

sky_area_degsq = 200.0
fsky = sky_area_degsq/mclh.FULL_SKY_AREA
vol_lightcone_mpc = fsky*(vol_z_max-vol_z_min)

args = (ran_key, lgmp_min, z_min, z_max, sky_area_degsq)
redshift_lightcone, lgmp_halopop_lightcone = mclh.mc_lightcone_host_halo_mass_function(*args)

In [ ]:
from diffsky.mass_functions import mc_hosts
redshift_singlez = np.median(redshift_lightcone)
args = (ran_key, lgmp_min, redshift_singlez, vol_lightcone_mpc)
lgmp_halopop_singlez = mc_hosts.mc_host_halos_singlez(*args)

In [ ]:
fig, ax = plt.subplots(1, 1)
yscale = ax.set_yscale('log')
xlabel = ax.set_xlabel(r'${\rm log_{10}M_{halo}}$')

counts_singlez, lgmp_bins, __ =ax.hist(lgmp_halopop_singlez, bins=100, alpha=0.7, label=r'${\rm single}$-$z\ {\rm halos}$')
counts_lightcone, __, __ =ax.hist(lgmp_halopop_lightcone, bins=lgmp_bins, alpha=0.7, label=r'${\rm lightcone\ halos}$')
leg = ax.legend()

In [ ]:
msk_well_sampled = counts_singlez > 200
counts_ratio = counts_lightcone[msk_well_sampled]/counts_singlez[msk_well_sampled]
assert np.allclose(counts_ratio, 1.0, atol=0.1)

## Check agreement between simulated and MC-generated halos

Check accuracy of Diffsky Monte Carlo generator by comparing to some pretabulated target data taken from the [SMDPL](https://www.cosmosim.org/metadata/smdpl/) simulation.

In [ ]:
import os
drn_testing_data = "../../diffsky/mass_functions/hmf_calibrations/tests/testing_data/"
logmp_bins_rockstar_z0 = np.load(os.path.join(drn_testing_data, "sfr_catalog_1.000000.hosthalos.logmp_bins.npy"))
lgcuml_density_rockstar_z0 = np.load(os.path.join(drn_testing_data, "sfr_catalog_1.000000.hosthalos.lgcuml_density.npy"))

logmp_bins_rockstar_z0 = np.load(os.path.join(drn_testing_data, "sfr_catalog_1.000000.hosthalos.logmp_bins.npy"))
lgcuml_density_rockstar_z0 = np.load(os.path.join(drn_testing_data, "sfr_catalog_1.000000.hosthalos.lgcuml_density.npy"))

In [ ]:
from diffsky.mass_functions.hmf_calibrations.smdpl_hmf import HMF_PARAMS as SMDPL_HMF_PARAMS

In [ ]:
smdpl_cosmo = DEFAULT_COSMOLOGY._replace(Om0=0.307115, h=0.67777)
Lbox_smdpl = 400.0/smdpl_cosmo.h
Vbox_smdpl = Lbox_smdpl**3

lgmp_min = logmp_bins_rockstar_z0.min()
z0 = 0.0
args = (ran_key, lgmp_min, z0, Vbox_smdpl)
lgmp_halopop_z0 = mc_hosts.mc_host_halos_singlez(*args, hmf_params=SMDPL_HMF_PARAMS)

mc_cuml_density_rockstar_z0 = np.array([np.sum(lgmp_halopop_z0>lgmp) for lgmp in logmp_bins_rockstar_z0])

In [ ]:
logmp_bins_rockstar_z1 = np.load(os.path.join(drn_testing_data, "sfr_catalog_0.460000.hosthalos.logmp_bins.npy"))
lgcuml_density_rockstar_z1 = np.load(os.path.join(drn_testing_data, "sfr_catalog_0.460000.hosthalos.lgcuml_density.npy"))
z1 = 1/0.460000-1

lgmp_min = logmp_bins_rockstar_z1.min()
args = (ran_key, lgmp_min, z1, Vbox_smdpl)
lgmp_halopop_z1 = mc_hosts.mc_host_halos_singlez(*args, hmf_params=SMDPL_HMF_PARAMS)

mc_cuml_density_rockstar_z1 = np.array([np.sum(lgmp_halopop_z1>lgmp) for lgmp in logmp_bins_rockstar_z1])

In [ ]:
fig, ax = plt.subplots(1, 1)

__=ax.loglog()

__=ax.plot(10**logmp_bins_rockstar_z0, 10**lgcuml_density_rockstar_z0, color='blue')
__=ax.plot(10**logmp_bins_rockstar_z0, mc_cuml_density_rockstar_z0/Vbox_smdpl, '--', color='blue')

__=ax.plot(10**logmp_bins_rockstar_z1, 10**lgcuml_density_rockstar_z1, color='red')
__=ax.plot(10**logmp_bins_rockstar_z1, mc_cuml_density_rockstar_z1/Vbox_smdpl, '--', color='red')

from matplotlib import lines as mlines
red_line=mlines.Line2D([],[],ls='-',c='red',label=r'$z=1$')
blue_line=mlines.Line2D([],[],ls='-',c='blue',label=r'$z=0$')
leg1=ax.legend(handles=[red_line, blue_line], loc='upper right')
__=ax.add_artist(leg1)

solid_line=mlines.Line2D([],[],ls='-',c='k',label=r'${\rm SMDPL\ simulation}$')
dashed_line=mlines.Line2D([],[],ls='--',c='k',label=r'${\rm Monte\ Carlo\ halos}$')
leg2=ax.legend(handles=[solid_line, dashed_line], loc='lower left') 

xlabel = ax.set_xlabel(r'$M_{\rm halo}\ [M_{\odot}]$')
ylabel = ax.set_ylabel(r'$n(>M_{\rm halo})\ {\rm [1/Mpc^3]}$')